In [6]:
torch.save(model.state_dict(), 'model.pth')

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from mnist_model import MNISTModel

def hook_fn(module, input, output, log_file):
    with open(log_file, 'a') as f:
        f.write(f"\nLayer: {module}")
        f.write(f"\nInput Tensor: {input}")
        f.write(f"\nOutput Tensor: {output}\n")

def register_hooks_for_all_layers(model, log_file):
    hooks = []
    for layer in model.children():
        hook = layer.register_forward_hook(lambda mod, inp, outp: hook_fn(mod, inp, outp, log_file))
        hooks.append(hook)
    return hooks

model = MNISTModel(custom_conv=False).float()
model.load_state_dict(torch.load('model.pth', weights_only=True))
model.eval()
log_file = "tensor_logs.txt"

hooks = register_hooks_for_all_layers(model, log_file)

torch.set_printoptions(profile="full")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
single_image, label = test_dataset[0]

single_image = single_image.unsqueeze(0)

with torch.no_grad():
    output = model(single_image)

for hook in hooks:
    hook.remove()

torch.set_printoptions(profile="default")
print(f"Tensors saved to {log_file}")


Tensors saved to tensor_logs.txt


In [16]:
from google.colab import files
files.download('tensor_logs.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>